In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle as pkl
import math
import requests
import csv
import sklearn
import json
import requests
from sklearn import preprocessing

In [2]:
def time_convert(time) :
      
    time = int(time)

    r1 = time %100

    if(r1<30) :
    
        time = time - r1
    
    elif(r1>=30) :
    
        time = time + (100 - r1)
    
    if(time == 2400):
    
        time = 0
    
    return time

In [3]:
#flight_data = pd.read_csv(r'C:\Users\badri\Desktop\ML-Project\Flight Data\Flight_Data.csv')
from pathlib import Path

# 1) Define your project root (where you launched the script / notebook)
BASE_DIR = Path().resolve()      # or: Path(__file__).parent if in a .py module

# 2) Point at your data folder
DATA_DIR = BASE_DIR / 'data'

# 3) Build each file’s path dynamically
flight_csv   = DATA_DIR / 'filtered_flights.csv'
weather_csv  = DATA_DIR / 'Weather' / 'Weather_Data.csv'
final_csv   = DATA_DIR / 'Flight_Weather.csv'

# 4) Read them with pandas
flight_data  = pd.read_csv(flight_csv)
weather_data = pd.read_csv(weather_csv)


In [4]:
flight_data.isnull().sum()

YEAR                     0
QUARTER                  0
MONTH                    0
DAY_OF_MONTH             0
FL_DATE                  0
ORIGIN_AIRPORT_ID        0
Origin                   0
DEST_AIRPORT_ID          0
CRS_DEP_TIME             0
DEP_TIME             60909
DEP_DELAY_NEW        60927
DEP_DEL15            60927
CRS_ARR_TIME             0
ARR_TIME             65147
ARR_DELAY_NEW        74908
ARR_DEL15            74908
dtype: int64

In [5]:

# Rename columns in flight_delay
flight_data = flight_data.rename(columns={
    'DAY_OF_MONTH': 'DayOfMonth',
    'FL_DATE': 'FlightDate',
    'ORIGIN_AIRPORT_ID': 'OriginAirportID',
    'DEST_AIRPORT_ID': 'DestAirportID',
    'CRS_DEP_TIME': 'CRSDepTime',
    'CRS_ARR_TIME': 'CRSArrTime',
    'DEP_TIME': 'DepTime',
    'ARR_TIME': 'ArrTime',
    'DEP_DEL15': 'DepDel15',
    'ARR_DEL15': 'ArrDel15',
    'ARR_DELAY_NEW': 'ArrDelayMinutes',
    'DEP_DELAY_NEW': 'DepDelayMinutes'
})

flight_data = flight_data[flight_data['ArrDelayMinutes'].notna()]

In [6]:
flight_data.isnull().sum()

YEAR               0
QUARTER            0
MONTH              0
DayOfMonth         0
FlightDate         0
OriginAirportID    0
Origin             0
DestAirportID      0
CRSDepTime         0
DepTime            0
DepDelayMinutes    0
DepDel15           0
CRSArrTime         0
ArrTime            0
ArrDelayMinutes    0
ArrDel15           0
dtype: int64

In [7]:
time_change = []

for i in flight_data['CRSDepTime'] :
    
    newtime = time_convert(i)
    
    time_change.append(newtime)

flight_data['DepartureTime'] = time_change

In [8]:
#weather_data = pd.read_csv(r'C:\Users\badri\Desktop\ML-Project\weather\Weather_Data.csv')

In [9]:
# …existing imports & DATA_DIR setup…

# 0) Parse/format dates exactly the same on both DataFrames
flight_data['FlightDate'] = pd.to_datetime(flight_data['FlightDate']) \
    .dt.strftime('%Y-%m-%d')
weather_data['Date']      = pd.to_datetime(weather_data['Date']) \
    .dt.strftime('%Y-%m-%d')

# 1) Ensure your “time” columns are ints in the same units (e.g. 000, 100, 200, …)
flight_data['DepartureTime'] = flight_data['DepartureTime'].astype(int)
weather_data['time']         = weather_data['time'].astype(int)

# 2) (Optional) Drop any weather‐rows whose time does not appear in flights
weather_data = weather_data[
    weather_data['time'].isin(flight_data['DepartureTime'])
]

# 3) Now do your outer‐merge diagnostic again
print("Dates in flights not in weather:",
      set(flight_data['FlightDate']) - set(weather_data['Date']))
print("Dates in weather not in flights:",
      set(weather_data['Date']) - set(flight_data['FlightDate']))
print("Times in flights not in weather:",
      set(flight_data['DepartureTime']) - set(weather_data['time']))
print("Times in weather not in flights:",
      set(weather_data['time']) - set(flight_data['DepartureTime']))

check = pd.merge(
    flight_data, weather_data,
    how='outer',
    left_on=['Origin','DepartureTime','FlightDate'],
    right_on=['Airport','time','Date'],
    indicator=True
)
print(check['_merge'].value_counts())

# 4) Finally your inner‐merge on the cleaned keys
final_dataframe = pd.merge(
    flight_data, weather_data, how='inner',
    left_on=['Origin','DepartureTime','FlightDate'],
    right_on=['Airport','time','Date']
).drop(['Origin','DepartureTime'], axis=1)

print("Merged shape:", final_dataframe.shape)

/var/folders/qd/hxnw7jb56lb72yft16bm17c40000gn/T/ipykernel_92475/4174796962.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  flight_data['FlightDate'] = pd.to_datetime(flight_data['FlightDate']) \


Dates in flights not in weather: set()
Dates in weather not in flights: set()
Times in flights not in weather: set()
Times in weather not in flights: set()
_merge
both          4980751
right_only      36296
left_only           0
Name: count, dtype: int64
Merged shape: (4980751, 34)


In [10]:
# …existing imports & DATA_DIR setup…

# 1) Make sure both sides have truly identical keys (strip whitespace, force same case)
flight_data['FlightDate'] = flight_data['FlightDate'].str.strip()
weather_data['Date']      = weather_data['Date'].str.strip()

flight_data['Origin']     = flight_data['Origin'].str.upper().str.strip()
weather_data['Airport']   = weather_data['Airport'].str.upper().str.strip()

# 2) Compare the sets of values on each key before merging
print("Dates in flights not in weather:", 
      set(flight_data['FlightDate']) - set(weather_data['Date']))
print("Dates in weather not in flights:", 
      set(weather_data['Date']) - set(flight_data['FlightDate']))

print("Airports in flights not in weather:", 
      set(flight_data['Origin']) - set(weather_data['Airport']))
print("Airports in weather not in flights:", 
      set(weather_data['Airport']) - set(flight_data['Origin']))

print("Times in flights not in weather:", 
      set(flight_data['DepartureTime']) - set(weather_data['time']))
print("Times in weather not in flights:", 
      set(weather_data['time']) - set(flight_data['DepartureTime']))

# 3) Do an outer‐merge with indicator to see which rows fail to match
check = pd.merge(
    flight_data, weather_data,
    how='outer',
    left_on=['Origin','DepartureTime','FlightDate'],
    right_on=['Airport','time','Date'],
    indicator=True
)
print(check['_merge'].value_counts())

# 4) Once you see where the mismatches are (e.g. wrong dates, missing hours or airports),
#    you can filter to the intersection and re-merge:
common_dates    = set(flight_data['FlightDate']).intersection(weather_data['Date'])
common_airports = set(flight_data['Origin']).intersection(weather_data['Airport'])
common_times    = set(flight_data['DepartureTime']).intersection(weather_data['time'])

fd = flight_data[
    flight_data['FlightDate'].isin(common_dates) &
    flight_data['Origin'].isin(common_airports) &
    flight_data['DepartureTime'].isin(common_times)
]
wd = weather_data[
    weather_data['Date'].isin(common_dates) &
    weather_data['Airport'].isin(common_airports) &
    weather_data['time'].isin(common_times)
]

final_dataframe = pd.merge(
    fd, wd, how='inner',
    left_on=['Origin','DepartureTime','FlightDate'],
    right_on=['Airport','time','Date']
).drop(['Origin','DepartureTime'], axis=1)

print("Merged shape:", final_dataframe.shape)

Dates in flights not in weather: set()
Dates in weather not in flights: set()
Airports in flights not in weather: set()
Airports in weather not in flights: set()
Times in flights not in weather: set()
Times in weather not in flights: set()
_merge
both          4980751
right_only      36296
left_only           0
Name: count, dtype: int64
Merged shape: (4980751, 34)


In [11]:
# …after you have built final_dataframe…

# 1) Quick overview: how many NaNs in each column?
nan_counts = final_dataframe.isnull().sum()
print(nan_counts[nan_counts > 0].sort_values(ascending=False))

# 2) List of columns that contain any NaN
cols_with_na = final_dataframe.columns[final_dataframe.isnull().any()].tolist()
print("Columns with NaN:", cols_with_na)

precipInches       4973215
visibilityMiles    4973215
pressureInches     4973215
uvIndex            4973215
dtype: int64
Columns with NaN: ['precipInches', 'visibilityMiles', 'pressureInches', 'uvIndex']


In [12]:
final_dataframe = pd.merge(flight_data,weather_data,how = 'inner',left_on = ['Origin','DepartureTime','FlightDate'],right_on = ['Airport','time','Date'])

final_dataframe = final_dataframe.drop(['Origin','DepartureTime'],axis = 1)
# …after you’ve computed cols_with_na…

#cols_with_na = ['precipInches',	'visibilityMiles',	'pressureInches',	'uvIndex']

# Option 1: drop all columns that contain any NaN
# final_dataframe = final_dataframe.dropna(axis=1, how='any')

# Option 2: drop only the columns you listed
final_dataframe = final_dataframe.drop(columns=cols_with_na)

# verify that no NaNs remain
print(final_dataframe.isnull().sum().any())  # should print

False


In [13]:
final_dataframe

,YEAR,QUARTER,MONTH,DayOfMonth,FlightDate,OriginAirportID,DestAirportID,CRSDepTime,DepTime,DepDelayMinutes,...,WindGustKmph,visibility,weatherCode,tempF,WindChillF,winddirDegree,humidity,time,Airport,Date
0,2016,1,1,1,2016-01-01,10397,10135,1341,1347.0,6.0,...,21,10,122,48,39,317,76,1400,ATL,2016-01-01
1,2016,1,1,1,2016-01-01,10397,10135,2100,2056.0,0.0,...,26,10,122,42,36,316,71,2100,ATL,2016-01-01
2,2016,1,1,1,2016-01-01,10397,10140,1740,1741.0,1.0,...,26,10,122,45,38,319,73,1800,ATL,2016-01-01
3,2016,1,1,1,2016-01-01,10397,10140,1930,1928.0,0.0,...,26,10,122,43,37,317,71,2000,ATL,2016-01-01
4,2016,1,1,1,2016-01-01,10397,10146,1340,1332.0,0.0,...,21,10,122,48,39,317,76,1400,ATL,2016-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4980746,2017,4,12,31,2017-12-31,14771,15041,945,935.0,0.0,...,8,10,116,52,51,220,85,1000,SFO,2017-12-31
4980747,2017,4,12,31,2017-12-31,14771,15304,1352,1344.0,0.0,...,6,10,119,54,53,320,89,1400,SFO,2017-12-31
4980748,2017,4,12,31,2017-12-31,14771,15376,820,817.0,0.0,...,8,10,116,51,50,142,86,800,SFO,2017-12-31
4980749,2017,4,12,31,2017-12-31,14771,15376,1600,1552.0,0.0,...,6,10,119,54,54,312,91,1600,SFO,2017-12-31


In [14]:
final_dataframe.to_csv(final_csv,index = False)